In [2]:
import torch

# GPU 사용 가능 여부 확인
is_available = torch.cuda.is_available()
print(f"GPU 사용 가능: {is_available}")

if is_available:
    # 사용 가능한 GPU 개수 및 이름 확인
    print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU 이름: {torch.cuda.get_device_name(0)}")


GPU 사용 가능: True
사용 가능한 GPU 개수: 1
현재 GPU 이름: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="PcTN9eTkDFtAnUWVIO12")
project = rf.workspace("vaibhav-jbwnp").project("ppe-hehno")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...


In [4]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 2) ultralytics 불러오기
from ultralytics import YOLO

# Roboflow가 내려받은 데이터셋 경로 (앞서 출력된 dataset.location 값 그대로 사용)
DATA_DIR = dataset.location          # 예: '/content/ppe-hehno-1'

# 3) 사전학습 가중치로 모델 초기화
model = YOLO("yolov8n.pt")           # 경량 모델 → 빠른 학습
# 처음부터 학습하려면 "yolov8n.yaml" 사용


In [17]:
# 추천 학습 코드
if __name__ == '__main__':
    model = YOLO('yolov8n.pt')
    model.train(
        data=f"{DATA_DIR}/data.yaml",
        epochs=100,
        imgsz=640,
        batch=24,
        device='0',
        workers=4,
        patience=30,      # 조기 종료 옵션 추가
        name="ppe_yolov8n"
    )


New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.203  Python-3.8.20 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Admin\capstone\PPE-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ppe_yolov8n

KeyboardInterrupt: 

In [ ]:
# 5) 검증(Validation) 성능 확인
metrics = model.val()                # mAP, Precision, Recall 등 자동 출력
print(metrics)


In [13]:
# 추론 실행
results = model.predict(
    source=f"{DATA_DIR}/test/images",
    conf=0.25,
    save=True,            # 주석 박스 그려진 이미지 저장
    imgsz=640,
    device = '0'
)

# 결과 객체는 list이므로 첫 번째 요소를 확인
first_res = results[0]                 # Results 객체
print("첫 번째 이미지 파일명:", first_res.path)
print("저장 폴더 경로      :", first_res.save_dir)



image 1/296 C:\Users\Admin\capstone\PPE-1\test\images\00004_jpg.rf.e74444023cd7ce5f4fdc565ae09d416e.jpg: 640x640 1 Person, 2 Safety_Vests, 26.6ms
image 2/296 C:\Users\Admin\capstone\PPE-1\test\images\00035_jpg.rf.3fc73679ce480c4896c7ab41503c8cf1.jpg: 640x640 1 Person, 1 Safety_Vest, 26.2ms
image 3/296 C:\Users\Admin\capstone\PPE-1\test\images\00090_jpg.rf.495bd93ec11e8b9e860da47787d1adca.jpg: 640x640 2 Hardhats, 3 Persons, 1 Safety_Boots, 1 Safety_Gloves, 1 Safety_Mask, 5 Safety_Vests, 30.5ms
image 4/296 C:\Users\Admin\capstone\PPE-1\test\images\00095_jpg.rf.b36499ea7ddf505365a448a7e1de22fc.jpg: 640x640 2 Persons, 4 Safety_Bootss, 5 Safety_Vests, 34.8ms
image 5/296 C:\Users\Admin\capstone\PPE-1\test\images\00178_jpg.rf.e14dc2b0f552cd9206e2f544df772aa8.jpg: 640x640 1 Hardhat, 6 Persons, 1 Safety_Boots, 2 Safety_Vests, 26.2ms
image 6/296 C:\Users\Admin\capstone\PPE-1\test\images\00272_jpg.rf.8a1276fbc44fe759d64daffc72b712d7.jpg: 640x640 1 Person, 2 Safety_Vests, 26.3ms
image 7/296 C:\Us

In [14]:
# 7) 최종 가중치 백업(선택)
import shutil, os
best_pt = os.path.join("runs/detect/ppe_yolov8n/weights/best.pt")
shutil.copy(best_pt, "ppe_best.pt")
print("best.pt 백업 완료 → ppe_best.pt")


best.pt 백업 완료 → ppe_best.pt
